# Limpieza de Datos

#### 1. Corregir Tipos de Datos
- Asegúrate de que cada columna tenga el tipo de dato correcto (fechas, números, categorías).

#### 2. Manejo de Valores Nulos
- Elimina o imputa (rellena) los valores nulos según el contexto y la importancia de la columna.

#### 3. Eliminar Duplicados
- Elimina filas duplicadas para evitar sesgos en el análisis.

#### 4. Corregir Inconsistencias
- Unifica formatos de texto (mayúsculas/minúsculas, espacios, tildes).
- Corrige errores ortográficos en categorías.

#### 5. Tratar Outliers
- Analiza los outliers detectados en el EDA y decide si eliminarlos, corregirlos o dejarlos.

#### 6. Crear Nuevas Variables (si aplica)
- Si necesitas variables derivadas (por ejemplo, año/mes de una fecha), créalas en esta etapa.

In [ ]:
import pandas as pd  # Data manipulation
import numpy as np  # Numerical operations
import matplotlib.pyplot as plt  # Data visualization
import seaborn as sns  # Data visualization
import plotly.express as px  # Interactive data visualization

import sys
sys.path.append('..')  # Ajusta el path si es necesario

from main import cargar_datos, guardar_datos

In [ ]:
# Cargar datos
datos = cargar_datos()

In [ ]:
datos.head()

## Desarrollo

#### 1. Corregir Tipos de Datos
- Asegúrate de que cada columna tenga el tipo de dato correcto (fechas, números, categorías).

In [ ]:
# Ver tipos de datos
datos.dtypes

**Interpretación**:

* Debemos cambiar el tipo de dato de *Date* al correcto

In [ ]:
# Cambiar tipo de dato de date
datos['Date'] = pd.to_datetime(datos['Date'], format='%d-%m-%y')
datos.head()

#### 2. Manejo de Valores Nulos
- Elimina o imputa (rellena) los valores nulos según el contexto y la importancia de la columna.

In [ ]:
datos.isnull().sum()

* *No hay NULOS*

#### 3. Eliminar Duplicados
- Elimina filas duplicadas para evitar sesgos en el análisis.

In [ ]:
duplicados = datos.duplicated().sum()
duplicados

if duplicados == 0:
    print("No hay duplicados en el DataFrame.")
else:
    print(f"Hay {duplicados} duplicados en el DataFrame.")
    datos = datos.drop_duplicates()
    print("Duplicados eliminados.")

#### 4. Corregir Inconsistencias
- Unifica formatos de texto (mayúsculas/minúsculas, espacios, tildes).
- Corrige errores ortográficos en categorías.

In [ ]:
# Unificar formatos de texto (mayúsculas/minúsculas, espacios, tildes) para los campos de texto
# Determinamos las columnas de tipo object
columnas_texto = datos.select_dtypes(include=['object']).columns
# Aplicamos la normalización a cada columna de texto
for col in columnas_texto:
    datos[col] = datos[col].str.title()  # Convertir a título (capitalizar la primera letra de cada palabra)
    datos[col] = datos[col].str.strip()  # Eliminar espacios al inicio y al final
    datos[col] = datos[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')  # Eliminar tildes
    

In [ ]:
datos.head()

#### 5. Tratar Outliers
- Analiza los outliers detectados en el EDA y decide si eliminarlos, corregirlos o dejarlos.

In [ ]:
# Identificar el número de outliers usando IQR
def identificar_outliers_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    outliers = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]
    return outliers
outliers_cantidad = identificar_outliers_iqr(datos, 'Total Sales')
outliers_cantidad

In [ ]:
# Aplicamos a todas las columnas y monstramos para cada una un array con los outliers
for col in datos.select_dtypes(include=[np.number]).columns:
    outliers = identificar_outliers_iqr(datos, col)
    print(f"Columna: {col}, Número de outliers: {len(outliers)}")
    print(outliers[[col]])
    

**Conclusión**:

* *No hay outliers aunque según nos muestre que hay 11 outliers en la columna `Total Sales`*
* *En conlusión según el IQR hay outliers pero son falsos outliers*

#### 6. Crear Nuevas Variables (si aplica)
- Si necesitas variables derivadas (por ejemplo, año/mes de una fecha), créalas en esta etapa.

In [ ]:
# Crear variables según la columna 'Date' y agruparlas en un tres nuevas columnas
datos['Year'] = datos['Date'].dt.year
datos['Month'] = datos['Date'].dt.strftime('%B')
datos['Day'] = datos['Date'].dt.day
datos.head()

In [ ]:
# Guardar datos limpios
guardar_datos(datos)